In [15]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [16]:
df = pd.read_csv('voice.csv')

In [17]:
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [18]:
df_sub = df[['label', 'mode', 'maxfun', 'maxdom', 'dfrange', 'sfm', 'modindx']]

Ik kies voor bovenstaande variabelen omdat deze allemaal op een manier te maken hebben met de luidheid en frequentie van een mannelijke of vrouwelijke stem. Iets waarvan ik vind dat later van pas komt voor een algoritme om te bepalen wat nou een mannelijke en een vrouwelijke stem is.

In [19]:
df_sub = df_sub.dropna()

In [20]:
dummies = pd.get_dummies(df_sub['label'])
df_sub_2 = pd.concat([df_sub, dummies], axis = 1)

Voer de dummies uit om label te scheiden in male en female.

In [21]:
df_sub_2.head()

,label,mode,maxfun,maxdom,dfrange,sfm,modindx,female,male
0,male,0.000000,0.275862,0.007812,0.000000,0.491918,0.000000,0,1
1,male,0.000000,0.250000,0.054688,0.046875,0.513724,0.052632,0,1
2,male,0.000000,0.271186,0.015625,0.007812,0.478905,0.046512,0,1
3,male,0.083878,0.250000,0.562500,0.554688,0.727232,0.247119,0,1
4,male,0.104261,0.266667,5.484375,5.476562,0.783568,0.208274,0,1


In [22]:
x = df_sub_2[['mode', 'maxfun', 'maxdom', 'dfrange', 'sfm', 'modindx']]
x = normalize(x)
y = df_sub_2[['male', 'female']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [23]:
knn = KNeighborsClassifier(n_neighbors=3) 
knn = knn.fit(x_train, y_train) 
knn.score(x_test, y_test)

0.7981072555205048

In [28]:
y_test_pred = knn.predict(x_test) 
cm = confusion_matrix(
    y_test.values.argmax(axis=1), y_test_pred.argmax(axis=1)) 
cm

array([[392, 102],
       [ 90, 367]], dtype=int64)

Hierboven staat de confussion matrix die hoort bij de gekozen kolummen in de subset. Deze zijn gebasseerd op de factoren of iemand man of vrouw is.

In [29]:
conf_matrix = pd.DataFrame(cm, index=['Man', 'Vrouw'], columns = ['Man_p', 'Vrouw_p']) 
conf_matrix

,Man_p,Vrouw_p
Man,392,102
Vrouw,90,367


In [30]:
accuracy = 759/951
print(accuracy)
precission = 392/482
print(precission)
recall = 392/494
print(recall)

0.7981072555205048
0.8132780082987552
0.7935222672064778


Zoals te zien is berekent het algoritme accuracy, precission en recall allemaal met bijna 80%. Dit betekent dus dat tijdens het uitvoeren van de knn testen het algoritme voor ongeveer 80% gelijk heeft als het gaat om het herkennen van mannen/vrouwen stemmen. Op de kaggle pagina stond echter dat er met een random forrest test tot 98% goed voorspeld kan worden. Dit laat dan ook de vraag of de knn wel zo toepasselijk is voor een onderzoek als dit.